# Machine learning for genetic data

In [ ]:
%pip install --upgrade --force-reinstall pillow

In [ ]:
%pylab inline 
# imports matplotlib as plt and numpy as np

In [ ]:
plt.rc('font', **{'size': 16}) # font size for text on plots

## Data description

* `data/athaliana_small.X.txt` is the design matrix. As many rows as samples, as many columns as SNPs
* the SNPs are given (in order) in `data/athaliana_small.snps.txt`. 
* the samples are given (in order) in `data/athaliana.samples.txt`.

* the transformed phenotypes are given in `data/athaliana.4W.pheno` and `data/athaliana.2W.pheno`. The first column is the sample's ID, and the second the phenotype.

* `data/athaliana.candidates.txt` contains a list of _A. thaliana_ genes known or strongly suspected to be associated with flowering times.

* the feature network is in `data/athaliana_small.W.txt`. It has been saved as 3 arrays, corresponding to the row, col, and data attributes of a [scipy.sparse coo_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html).

## Loading the data
We will start by working without the feature network, on the 2W phenotype.

In [ ]:
# Load the SNP names
with open('data/athaliana_small.snps.txt') as f:
    snp_names = f.readline().split()
    f.close()
print(len(snp_names))

In [ ]:
# Load the design matrix -- this can take time!
X = np.loadtxt('data/athaliana_small.X.txt',  # file names
               dtype = 'int') # values are integers

In [ ]:
X.shape

In [ ]:
p = X.shape[1]

We have 1307 samples and 9 419 SNPs.

The 2W phenotype is not available for all samples. 

#### Load the sample names

In [ ]:
samples = list(np.loadtxt('data/athaliana.samples.txt', # file names
                         dtype = int)) # values are integers
print(len(samples))
print(samples[:10])

### Load the 2W phenotype data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/athaliana.2W.pheno', # file name
                 header = None, # columns have no header
                 delim_whitespace = True) # columns are separated by white space

# Create vector of sample IDs
samples_with_phenotype = list(df[0])
print(len(samples_with_phenotype), "samples have a phenotype")

# Create vector of phenotypes
y_2W = np.array(df[1])

In [ ]:
# Restrict X to the samples with a 2W phenotype, in correct order
# X_2W[i] = X[samples.index(samples_with_phenotype[i])]
X_2W = X[np.array([samples.index(sample_id) \
                   for sample_id in samples_with_phenotype]), :]
print(X_2W.shape)

### Load the list of candidate genes

In [ ]:
with open('data/athaliana.candidates.txt') as f:
    candidate_genes = f.readline().split()
    f.close()

### Load the snps to gene mapping

In [ ]:
genes_by_snp = {} # key: SNP, value = [genes in/near which this SNP is]
with open('data/athaliana.snps_by_gene.txt') as f:
    for line in f:
        ls = line.split()
        gene_id = ls[0]
        for snp_id in ls[1:]:
            if not snp_id in genes_by_snp:
                genes_by_snp[snp_id] = []
            genes_by_snp[snp_id].append(gene_id) 

### Split the data in a train and test set

We will set aside a test set, containing 20% of our samples, on which to evaluate the quality of our predictive models.

In [ ]:
from sklearn import model_selection

In [ ]:
X_2W_tr, X_2W_te, y_2W_tr, y_2W_te = \
    model_selection.train_test_split(X_2W, y_2W, test_size = 0.2, random_state = 17)
print(X_2W_tr.shape, X_2W_te.shape)

## Elastic net

One solution to make the lasso more stable is to use a combination of the l1 and l2 regularizations.

We are now minimizing the loss + a linear combination of an l1-norm and an l2-norm over the regression weights. This imposes sparsity, but encourages correlated features to be selected together, where the lasso would tend to pick only one (at random) of a group of correlated features.

The elastic net is implemented in scikit-learn's [linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet).

### Regularization path

In [ ]:
from sklearn import linear_model

In [ ]:
eps = 3e-1

In [ ]:
alphas_lasso, coefs_lasso, _ = linear_model.lasso_path(X_2W_tr[:, :], y_2W_tr, eps = eps, fit_intercept = True)

In [ ]:
alphas_enet, coefs_enet, _ = linear_model.enet_path(
    X_2W_tr[:, :], y_2W_tr, eps = eps, l1_ratio = 0.95, fit_intercept = True)

In [ ]:
from itertools import cycle
import matplotlib.colors as mcolors
colors = cycle(list(mcolors.TABLEAU_COLORS.keys()))
figure(figsize = (10, 6))
neg_log_alphas_lasso = -np.log10(alphas_lasso)
neg_log_alphas_enet = -np.log10(alphas_enet)
for coef_l, coef_e, c in zip(coefs_lasso, coefs_enet, colors):
    l1 = plt.plot(neg_log_alphas_lasso, coef_l, c = c)
    l2 = plt.plot(neg_log_alphas_enet, coef_e, linestyle = '--', c = c)

plt.xlabel('-Log(alpha)')
plt.ylabel('coefficients')
plt.title('Lasso and Elastic-Net Paths')
plt.legend((l1[-1], l2[-1]), ('Lasso', 'Elastic-Net'), loc='lower left')
plt.axis('tight')

__Q: Compared to the lasso, what is the effect of the elastic-net on the coefficients?__

### Fit lastic-net

In [ ]:
from sklearn import linear_model, metrics

In [ ]:
# Parameters grid
alphas = np.logspace(-4., 1., num = 15)
ratios = np.linspace(0.5, 1., num = 15)

__Q: Define the elastic net model (call it `model_l1l2`) using the functions `ElasticNet` and `GridSearchCV`.__

In [ ]:
## Answer:

In [ ]:
plt.figure(figsize = (6, 4))
plt.scatter(range(p), # x = SNP position
            model_l1l2.coef_)  # y = regression weights

plt.xlabel("SNP")
plt.ylabel("elastic net regression weight")
plt.xlim([0, p])

__How can you interpret these results? How many SNPs contribute to explaining the phenotype?__

In [ ]:
print("%d SNPs selected," % \
    np.nonzero(model_l1l2.coef_)[0].shape)

candidate_genes_hit = set([])
num_snps_in_candidate_genes = 0
for snp_idx in np.nonzero(model_l1l2.coef_)[0]:
    for gene_id in genes_by_snp[snp_names[snp_idx]]:
        if gene_id in candidate_genes and gene_id not in candidate_genes_hit:
            candidate_genes_hit.add(gene_id)
            num_snps_in_candidate_genes += 1

print("of which %d are in %d candidate genes" % (num_snps_in_candidate_genes, 
                                                          len(candidate_genes_hit)))

### Predictive power 

In [ ]:
y_2W_l1l2_pred = model_l1l2.predict(X_2W_te)

print("Percentage of variance explained (using %d SNPs): %.2f" % \
     (np.nonzero(model_l1l2.coef_)[0].shape[0], 
      metrics.explained_variance_score(y_2W_te, y_2W_l1l2_pred)))

In [ ]:
plt.figure(figsize = (4, 4))
plt.scatter(y_2W_te, y_2W_l1l2_pred)

plt.xlabel("true phenotype")
plt.ylabel("prediction")
plt.xlim([np.min(y_2W_te) - 0.05, np.max(y_2W_te) + 0.05])
plt.ylim([np.min(y_2W_te) - 0.05, np.max(y_2W_te) + 0.05])
plt.axline(xy1 = [0, 0], slope = 1, c = "black")

## Multi-task feature selection

1) Repeat the previous analysis for the 4W phenotype. It is very similar to the 2W phenotype, except that the seeds have been vernelized for 4 weeks. 

2) It is not unreasonable to expect the genomic regions driving both those phenotypes to be (almost) the same. Use the multi-task version of the Lasso, ENet, or ncLasso algorithms to analyzed both phenotypes simultaneously.

Use [sklearn.linear_model.MultiTaskLasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.MultiTaskLasso.html#sklearn.linear_model.MultiTaskLasso) + [User Guide](http://scikit-learn.org/stable/auto_examples/linear_model/plot_multi_task_lasso_support.html)

3) __Q: Is it the same "multi-task" setting as the one in the lecture? What is the difference?__

### Load 4W and 2W phenotypes

In [ ]:
X_2W = X[np.array([samples.index(sample_id) \
                   for sample_id in samples_with_phenotype]), :]

In [ ]:
df_2W = pd.read_csv('data/athaliana.2W.pheno', # file name
                 header = None, # columns have no header
                 delim_whitespace = True) # columns are separated by white space

# Create vector of sample IDs
samples_with_phenotype_2W = list(df[0])
print(len(samples_with_phenotype_2W), "samples have a phenotype")

# Create vector of phenotypes
y_2W = np.array(df_2W[1])

In [ ]:
X = np.loadtxt('data/athaliana_small.X.txt',  # file names
               dtype = 'int') # values are integers

In [ ]:
df_4W = pd.read_csv('data/athaliana.4W.pheno', # file name
                 header = None, # columns have no header
                 delim_whitespace = True) # columns are separated by white space

# Create vector of sample IDs
samples_with_phenotype_4W = list(df_4W[0])
print(len(samples_with_phenotype_4W), "samples have a 4W phenotype")

# Create vector of phenotypes
y_4W = np.array(df_4W[1])

In [ ]:
X_4W = X[np.array([samples.index(sample_id) \
                   for sample_id in samples_with_phenotype_4W]), :]

In [ ]:
X_4W_tr, X_4W_te, y_4W_tr, y_4W_te = \
    model_selection.train_test_split(X_4W, y_4W, test_size = 0.2, random_state = 17)
print(X_4W_tr.shape, X_4W_te.shape)

### Preliminary analysis
__Q: Why would we expect the 2W and 4W phenotypes to share many explanatory SNPs? How could you verify this using Manhattan plots?__

In [ ]:
## Answer:
## Tip: compute pvalues for both 2W and 4W (using association test, cf practical1.ipynb).
## Tip: plot both these p-values and comment the visual result.
import statsmodels.api as sm

Another possible visualization is to plot both pvalues ax xy-coordinates:

In [ ]:
# We can now plot the pvalues of 2W and 4W:
figure(figsize = (5, 5))
#plt.scatter(-np.log10(pvalues_2W), -np.log10(pvalues_4W))
plt.xlabel('2W'); plt.ylabel('4W')

__Q: What conclusion can you draw? Do the 2W and 4W phenotypes seem to be linked to the same genome loci?__

### Define design matrix

Sample with *both* 2W and 4W phenotypes.

In [ ]:
samples_with_phenotype_both = list(set(samples_with_phenotype_2W).intersection(samples_with_phenotype_4W))
print(len(samples_with_phenotype_both), "samples have both phenotypes")

In [ ]:
X_both = X[np.array([samples.index(sample_id) for sample_id in samples_with_phenotype_both]), :]
del X # You can delete X now if you want, to free space

Filter, order, and merge the two phenotype data frames

In [ ]:
## Filter out the samples that do not have both phenotypes
df_4W_filtered = df_4W[df_4W[0].isin(samples_with_phenotype_both)]
df_2W_filtered = df_2W[df_2W[0].isin(samples_with_phenotype_both)]
## Set sample names as the DataFrame index name
df_2W_filtered.index = list(df_2W_filtered[0])
df_4W_filtered.index = list(df_4W_filtered[0])
## Reorder both datasets so as to have the phenotypes in the same order
df_2W_ordered = df_2W_filtered.reindex(samples_with_phenotype_both)
df_4W_ordered = df_4W_filtered.reindex(samples_with_phenotype_both)
## Define the phenotype matrix
y_both = np.array([df_2W_ordered[1], df_4W_ordered[1]]).T

Split in train and test sets

In [ ]:
X_both_tr, X_both_te, y_both_tr, y_both_te = \
    model_selection.train_test_split(X_both, y_both, test_size = 0.2, random_state = 19)

In [ ]:
print(y_both.shape, y_both_tr.shape, y_both_te.shape)
print(X_both.shape, X_both_tr.shape, X_both_te.shape)

### Run Multi-task lasso

In [ ]:
from sklearn import linear_model, metrics

__Q:__ Run MultiTaskLasso (and call the object `model_mt_l1`).

In [ ]:
# Answer:

In [ ]:
metrics.explained_variance_score(y_both_te, model_mt_l1.best_estimator_.predict(X_both_te))

In [ ]:
model_mt_l1_2 = linear_model.MultiTaskLassoCV(fit_intercept = True, max_iter = 6000, 
                                              cv = 5, random_state = 18).fit(X_both_tr, y_both_tr)

In [ ]:
metrics.explained_variance_score(y_both_te, model_mt_l1_2.predict(X_both_te))

In [ ]:
plt.figure(figsize = (8, 5))
#plt.spy(ml_lasso_cv.best_estimator_.coef_)
plt.scatter(range(p), model_mt_l1.best_estimator_.coef_[0, :])
plt.scatter(range(p), model_mt_l1.best_estimator_.coef_[1, :])

### Predictive power

In [ ]:
from sklearn import metrics

In [ ]:
y_both_l1_pred = model_mt_l1.best_estimator_.predict(X_both_te)

print("Percentage of variance explained (using %d SNPs): %.2f" % \
     (np.nonzero(model_mt_l1.best_estimator_.coef_)[0].shape[0], 
      metrics.explained_variance_score(y_both_te, y_both_l1_pred)))

### Run Multi-task elastic-net
See the (https://scikit-learn.org/stable/modules/linear_model.html#multi-task-elastic-net)[user guide] and (API)[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.MultiTaskElasticNet.html#sklearn.linear_model.MultiTaskElasticNet].

__Q: Do the same as before, but with (multi-task) elastic net instead!__

In [ ]:
# Answer:

## Network-constrained lasso

### Load the network

In [ ]:
from scipy import sparse

In [ ]:
w_saved = np.loadtxt('data/athaliana_small.W.txt') # adjacency matrix is in coordinate (or triplet) format

In [ ]:
# Create adjacency matrix in scipy.sparse.coo_matrix format
W = sparse.coo_matrix((w_saved[2, :], (np.array(w_saved[0, :], dtype = int), 
                                       np.array(w_saved[1, :], dtype = int))),
                      shape = (p, p))

### Create the transposed incidence matrix

__Q: The incidence matrix of a graph? What is it again? You may refer to Wikipedia, or the paper `Network-constrained regularization and variable selection
for analysis of genomic data`__

Now let's create it.

In [ ]:
degrees = np.zeros((p, ))
for vertex in W.row:
    degrees[vertex] += 2

In [ ]:
row_ix = 0
tim = sparse.lil_matrix((w_saved[0].shape[0] // 2, p))
for ix, edge in enumerate(w_saved[2, :]):
    if w_saved[0, ix] < w_saved[1, ix]: # counts each edge-vertex link only once
        tim[row_ix, w_saved[0, ix]] = np.sqrt(edge / degrees[W.row[ix]])
        tim[row_ix, w_saved[1, ix]] = -np.sqrt(edge / degrees[W.row[ix]])
        row_ix += 1

### Define ncLasso class

In [ ]:
from sklearn import base, linear_model

The network-constrained lasso is just a disguised lasso! By defining a modified design matrix X_new and output vector y_new, we can solve the network-constrained lasso problem using the usual `linear_model.Lasso` function.

__Q: Go on the [paper introducing network-constrained lasso](https://watermark.silverchair.com/btn081.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAArowggK2BgkqhkiG9w0BBwagggKnMIICowIBADCCApwGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMDe8GLDpW7r7m-akpAgEQgIICbfpkYq0Lr-mTYblz7p4QoXY1GEQWv79yv4os3nGyI9V7jpY4hitfcXBPLHoskH-EGIbvKa_R4cfMBrKOPpB3v_CJDz_fjBCaLC2MHeW9R4hXKbvIWQETPQbBU5mUBbZ9xgi4qaFcumEW8huUcwZoOIH2lWE2FshQmDNGsc7M0sQmVUPh2I6EQ3kh3lDA6_MGVwJ2rFgc7celLZDPOsVCMNQDf33ZcZTxZukzZINIndNpt6TwP4OCBWaSgRnBChfic9mhwAqxaZ6if-mGGMbgaqCkoWo5LIkOd6uqAAOoD7G3hzDnqimAauDAaUEgK2UTIV6_xFLC3lmjuGWoXrih4pM_6vsgTjOVj59iaPDbd6a0IpnpmzkJdlG3eNmvah4UPv02MGJZvDda0GNrySaTSvNrGtkzXgGpIe_nIQW75H7xI0n8yRvtLPQ2aph22U2LvSpBAyxw23-oW13K5PD9irGviaJ9pNbQShxvFd_tFToCy0J2lnzzgp-JIsQJlJzXbwtePqXH3VgakG9YjCRapJCU0fC8gnbwzjUp-g-P2NpjBYx7Ktq5TDY19vC1bFUOqsBAJ3cmQpCES5bc3llSHRASkmQxl7h9VpGWGm5xv98_hHdl3Ee3G63XUIBit5717BWXP-nlLM_4JrnDXbXe2MJjViSQCW7YNcSFtQZ3DlsGM7Ygoz-pSs5ZbTUDOQq8AO6nniBKekcBnWSV5zFHbZsJaCKccMySMYxyyeM-9oS_YLZ09Yo4e9cGO8HwlDtV6bCCoz714n4oX0XUqDFFv8Ux5qXgvdEMGTYUZl1cUBLvO8Is2nWAq6cP6Hn86w) and fill in the missing line using Lemma 1.__

In [ ]:
class ncLasso(base.BaseEstimator, base.RegressorMixin):
    def __init__(self, transposed_incidence = None, lambda1 = 1.0, lambda2 = 1.0):
        self.transposed_incidence = transposed_incidence # sparse matrix
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        
    def fit(self, X, y, max_iter = 1000):       
        alpha = self.lambda1 / (np.sqrt(self.lambda2 + 1.))
        self.lasso = linear_model.Lasso(fit_intercept = True, 
                                        alpha = alpha, max_iter = max_iter)
        y_new = np.hstack((y, np.zeros((self.transposed_incidence.shape[0], ))))
        X_new = 1 / np.sqrt(self.lambda2 + 1) * sparse.vstack(
            ## Missing line
        )
        self.lasso.fit(X_new, y_new)
        self.coef_ = self.lasso.coef_[:X.shape[1]] / np.sqrt(self.lambda2 + 1)
        return self
        
        
    def predict(self, X, y = None):
        return self.lasso.predict(X)
    
    
    def score(self, X, y = None):
        return self.lasso.score(X, y)                                        

### Run network-constrained lasso

In [ ]:
model_nclasso = ncLasso(tim, 0.00005, 0.00001)

import time
start = time.time()
## DO NOT RUN (takes two hours to run)
# model_nclasso.fit(X_2W_tr, y_2W_tr, max_iter = 2000)
## END DO NOT RUN
end = time.time()
#print(end - start)

In [ ]:
import pickle
## DO NOT RUN
#with open('model_nclasso.obj', 'wb') as f:
#    pickle.dump(model_nclasso, f, pickle.HIGHEST_PROTOCOL)
## END DO NOT RUN

with open('model_nclasso.obj', 'rb') as f:
    model_nclasso = pickle.load(f)

In [ ]:
plt.figure(figsize = (6, 4))
plt.scatter(range(p), # x = SNP position
            model_nclasso.coef_)  # y = regression weights

plt.xlabel("SNP")
plt.ylabel("elastic net regression weight")
plt.xlim([0, p])

In [ ]:
print("%d SNPs selected," % \
    np.nonzero(model_nclasso.coef_)[0].shape)

candidate_genes_hit = set([])
num_snps_in_candidate_genes = 0
for snp_idx in np.nonzero(model_nclasso.coef_)[0]:
    for gene_id in genes_by_snp[snp_names[snp_idx]]:
        if gene_id in candidate_genes:
            candidate_genes_hit.add(gene_id)
            num_snps_in_candidate_genes += 1

print("%d SNPs are in %d candidate genes" % (num_snps_in_candidate_genes, 
                                                          len(candidate_genes_hit)))

In [ ]:
y_ncl1_pred = model_nclasso.predict(X_2W_te)
print(metrics.explained_variance_score(y_2W_te, y_ncl1_pred))

## Going further
You can also used methods developped specifically for SNP detections! 

For instance SConES (Azencott et al 2013) has a publicly available implementation as an R package (https://github.com/hclimente/martini) or in python (https://github.com/chagaz/sfan).